In [1]:
%reload_ext nb_black
import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy import stats
from scipy.stats import bartlett
from scipy.stats import levene


import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
import warnings

from sqlalchemy import create_engine
warnings.filterwarnings("ignore")
sns.set(style="whitegrid")



<IPython.core.display.Javascript object>

In [2]:
def missingness_summary(df, print_log=False, sort="none"):
    summary = df.apply(lambda x: x.isna().sum() / x.shape[0])

    if print_log == True:
        if sort == "none":
            print(summary)
        elif sort == "ascending":
            print(summary.sort_values())
        elif sort == "descending":
            print(summary.sort_values(ascending=False))
        else:
            print("Invalid value for sort parameter.")

    return

<IPython.core.display.Javascript object>

# Weather Model

* First, load the dataset from the weatherinszeged table from Thinkful's database.
* Like in the previous checkpoint, build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed. Now, estimate your model using OLS. What are the R-squared and adjusted R-squared values? Do you think they are satisfactory? Why?
* Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Now, what is the R-squared of this model? Does this model improve upon the previous one?
* Add visibility as an additional explanatory variable to the first model and estimate it. Did R-squared increase? What about adjusted R-squared? Compare the differences put on the table by the interaction term and the visibility in terms of the improvement in the adjusted R-squared. Which one is more useful?
* Choose the best one from the three models above with respect to their AIC and BIC scores. Validate your choice by discussing your justification with your mentor.

In [3]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "weatherinszeged"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)
weatherinszeged_df = pd.read_sql_query("select * from weatherinszeged", con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

<IPython.core.display.Javascript object>

In [4]:
weatherinszeged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   date                 96453 non-null  datetime64[ns, UTC]
 1   summary              96453 non-null  object             
 2   preciptype           96453 non-null  object             
 3   temperature          96453 non-null  float64            
 4   apparenttemperature  96453 non-null  float64            
 5   humidity             96453 non-null  float64            
 6   windspeed            96453 non-null  float64            
 7   windbearing          96453 non-null  float64            
 8   visibility           96453 non-null  float64            
 9   loudcover            96453 non-null  float64            
 10  pressure             96453 non-null  float64            
 11  dailysummary         96453 non-null  object             
dtypes: datetime64[ns, 

<IPython.core.display.Javascript object>

In [5]:
y = weatherinszeged_df["temperature"] - weatherinszeged_df["apparenttemperature"]
X = weatherinszeged_df[["humidity", "windspeed"]]

<IPython.core.display.Javascript object>

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

lr1_results = sm.OLS(y_train, X_train_const).fit()

<IPython.core.display.Javascript object>

In [7]:
lr1_results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.291
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                 1.483e+04
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        21:19:00   Log-Likelihood:            -1.2760e+05
No. Observations:               72339   AIC:                         2.552e+05
Df Residuals:                   72336   BIC:                         2.552e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.4471      0.024   -101.379      0.000      -2.494      -2.400
humidity       3.0395      0.028    110.355      0.000       2.986       3.093
windspeed      0.1193      0.001    153.414      0.000       0.118       0.121
==============================================================================
Omnibus:                     2874.768   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3383.791
Skew:                           0.467   Prob(JB):                         0.00
Kurtosis:                       3.501   Cond. No.                         88.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<IPython.core.display.Javascript object>

* Both $R^2$ and adj$R^2$ are small, at 0.291. Our model barely predicts 30% of the variance in the target

* Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Now, what is the R-squared of this model? Does this model improve upon the previous one?

In [8]:
X["humidity_x_windspeed"] = X["humidity"] * X["windspeed"]

<IPython.core.display.Javascript object>

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

lr2_results = sm.OLS(y_train, X_train_const).fit()

<IPython.core.display.Javascript object>

In [10]:
lr2_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.344
Model:                            OLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                 1.265e+04
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        21:20:42   Log-Likelihood:            -1.2478e+05
No. Observations:               72339   AIC:                         2.496e+05
Df Residuals:                   72335   BIC:                         2.496e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.1074      0.038     -2.801      0.005      -0.183      -0.032
humidity                -0.1478      0.049     -2.997      0.003      -0.244      -0.051
windspeed               -0.0891      0.003    -31.598      0.000      -0.095      -0.084
humidity_x_windspeed     0.2950      0.004     76.627      0.000       0.287       0.303
==============================================================================
Omnibus:                     3665.198   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7362.002
Skew:                           0.366   Prob(JB):                         0.00
Kurtosis:                       4.381   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<IPython.core.display.Javascript object>

* Creating an interaction of windspeed and humidity made the model stronger, but it is still performing poorly

* Add visibility as an additional explanatory variable to the first model and estimate it. Did R-squared increase? What about adjusted R-squared? Compare the differences put on the table by the interaction term and the visibility in terms of the improvement in the adjusted R-squared. Which one is more useful?

In [11]:
X["visibility"] = weatherinszeged_df["visibility"]
X

,humidity,windspeed,humidity_x_windspeed,visibility
0,0.89,14.1197,12.566533,15.8263
1,0.86,14.2646,12.267556,15.8263
2,0.89,3.9284,3.496276,14.9569
3,0.83,14.1036,11.705988,15.8263
4,0.83,11.0446,9.167018,15.8263
...,...,...,...,...
96448,0.43,10.9963,4.728409,16.1000
96449,0.48,10.0947,4.845456,15.5526
96450,0.56,8.9838,5.030928,16.1000
96451,0.60,10.5294,6.317640,16.1000


<IPython.core.display.Javascript object>

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

lr2_results = sm.OLS(y_train, X_train_const).fit()

<IPython.core.display.Javascript object>

In [13]:
lr2_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.366
Model:                            OLS   Adj. R-squared:                  0.366
Method:                 Least Squares   F-statistic:                 1.044e+04
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        21:28:03   Log-Likelihood:            -1.2355e+05
No. Observations:               72339   AIC:                         2.471e+05
Df Residuals:                   72334   BIC:                         2.472e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    1.0651      0.044     23.973      0.000       0.978       1.152
humidity                -0.8515      0.050    -16.864      0.000      -0.950      -0.753
windspeed               -0.1019      0.003    -36.594      0.000      -0.107      -0.096
humidity_x_windspeed     0.3140      0.004     82.530      0.000       0.307       0.321
visibility              -0.0639      0.001    -49.919      0.000      -0.066      -0.061
==============================================================================
Omnibus:                     4026.097   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9112.589
Skew:                           0.362   Prob(JB):                         0.00
Kurtosis:                       4.581   Cond. No.                         245.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<IPython.core.display.Javascript object>

# House Prices Model
* Load the houseprices data from Thinkful's database.
* Run your house prices model again and interpret the results. Which features are statistically significant, and which are not?
* Now, exclude the insignificant features from your model. Did anything change?
* Interpret the statistically significant coefficients by quantifying their relations with the house prices. Which features have a more prominent effect on house prices?
* Do the results sound reasonable to you? If not, try to explain the potential reasons.









In [43]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "houseprices"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)
houseprices = pd.read_sql_query("select * from houseprices", con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
houseprices.dropna()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
missingness_summary(houseprices, print_log=True, sort="descending")

poolqc           0.995205
miscfeature      0.963014
alley            0.937671
fence            0.807534
fireplacequ      0.472603
                   ...   
centralair       0.000000
salecondition    0.000000
heating          0.000000
totalbsmtsf      0.000000
id               0.000000
Length: 81, dtype: float64


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:
houseprices = houseprices.drop(
    columns=["poolqc", "miscfeature", "alley", "fence", "fireplacequ", "lotfrontage"]
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
missingness_summary(houseprices, print_log=True, sort="descending")

garagetype       0.055479
garageyrblt      0.055479
garagefinish     0.055479
garagecond       0.055479
garagequal       0.055479
                   ...   
centralair       0.000000
heatingqc        0.000000
salecondition    0.000000
totalbsmtsf      0.000000
id               0.000000
Length: 75, dtype: float64


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
houseprices = houseprices.drop(
    columns=["garagetype", "garageyrblt", "garagefinish", "garagecond", "garagequal"]
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [49]:
houseprices = houseprices.dropna()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [50]:
houseprices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1412 entries, 0 to 1459
Data columns (total 70 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1412 non-null   int64  
 1   mssubclass     1412 non-null   int64  
 2   mszoning       1412 non-null   object 
 3   lotarea        1412 non-null   int64  
 4   street         1412 non-null   object 
 5   lotshape       1412 non-null   object 
 6   landcontour    1412 non-null   object 
 7   utilities      1412 non-null   object 
 8   lotconfig      1412 non-null   object 
 9   landslope      1412 non-null   object 
 10  neighborhood   1412 non-null   object 
 11  condition1     1412 non-null   object 
 12  condition2     1412 non-null   object 
 13  bldgtype       1412 non-null   object 
 14  housestyle     1412 non-null   object 
 15  overallqual    1412 non-null   int64  
 16  overallcond    1412 non-null   int64  
 17  yearbuilt      1412 non-null   int64  
 18  yearremo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [52]:
# used code from answer key for times sake

houseprices = pd.concat(
    [
        houseprices,
        pd.get_dummies(houseprices.mszoning, prefix="mszoning", drop_first=True),
    ],
    axis=1,
)
houseprices = pd.concat(
    [houseprices, pd.get_dummies(houseprices.street, prefix="street", drop_first=True)],
    axis=1,
)
dummy_column_names = list(
    pd.get_dummies(houseprices.mszoning, prefix="mszoning", drop_first=True).columns
)
dummy_column_names = dummy_column_names + list(
    pd.get_dummies(houseprices.street, prefix="street", drop_first=True).columns
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [65]:
# Y is the target variable
Y = houseprices["saleprice"]
# X is the feature set
X = houseprices[
    ["overallqual", "grlivarea", "garagecars", "garagearea", "totalbsmtsf"]
    + dummy_column_names
]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     457.9
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        20:55:43   Log-Likelihood:                -16907.
No. Observations:                1412   AIC:                         3.384e+04
Df Residuals:                    1401   BIC:                         3.389e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.195e+05   1.82e+04     -6.582      0.000   -1.55e+05   -8.39e+04
overallqual  2.358e+04   1121.150     21.032      0.000    2.14e+04    2.58e+04
grlivarea      45.5603      2.534     17.976      0.000      40.588      50.532
garagecars   1.375e+04   3055.048      4.499      0.000    7752.690    1.97e+04
garagearea     14.4560     10.591      1.365      0.173      -6.320      35.232
totalbsmtsf    31.0437      3.265      9.507      0.000      24.638      37.449
mszoning_FV  1.217e+04   6903.187      1.763      0.078   -1369.598    2.57e+04
mszoning_FV  1.217e+04   6903.187      1.763      0.078   -1369.598    2.57e+04
mszoning_RH  6297.9655   8063.249      0.781      0.435   -9519.378    2.21e+04
mszoning_RH  6297.9655   8063.249      0.781      0.435   -9519.378    2.21e+04
mszoning_RL  1.334e+04   6401.726      2.084      0.037     780.471    2.59e+04
mszoning_RL  1.334e+04   6401.726      2.084      0.037     780.471    2.59e+04
mszoning_RM  4089.8879   6480.049      0.631      0.528   -8621.757    1.68e+04
mszoning_RM  4089.8879   6480.049      0.631      0.528   -8621.757    1.68e+04
street_Pave  -153.0119   8185.592     -0.019      0.985   -1.62e+04    1.59e+04
street_Pave  -153.0119   8185.592     -0.019      0.985   -1.62e+04    1.59e+04
==============================================================================
Omnibus:                      432.931   Durbin-Watson:                   1.971
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            45506.112
Skew:                          -0.323   Prob(JB):                         0.00
Kurtosis:                      30.804   Cond. No.                     3.10e+20
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.96e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

* Insignificant features: garagearea, all mszoning fields, and streetpave
* Overallqual: each unit results in a $23,580 increase in house price
* grlivarea: each unit results in a $45.56 increase in house price
* garagecars: each unit results in a $13,750 increase in house price
* totalbsmtsf: each unit results in a $31.04 increase in house price


In [66]:
insig_col = [
    "garagearea",
    "mszoning_FV",
    "mszoning_FV",
    "mszoning_RH",
    "mszoning_RH",
    "mszoning_RM",
    "mszoning_RM",
    "street_Pave",
    "street_Pave",
]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [67]:
X = X.drop(columns=insig_col)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.763
Method:                 Least Squares   F-statistic:                     909.6
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        20:56:09   Log-Likelihood:                -16912.
No. Observations:                1412   AIC:                         3.384e+04
Df Residuals:                    1406   BIC:                         3.387e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.119e+05   4986.723    -22.432      0.000   -1.22e+05   -1.02e+05
overallqual  2.422e+04   1091.004     22.202      0.000    2.21e+04    2.64e+04
grlivarea      45.2722      2.521     17.955      0.000      40.326      50.218
garagecars   1.744e+04   1788.309      9.750      0.000    1.39e+04    2.09e+04
totalbsmtsf    31.6021      3.182      9.932      0.000      25.361      37.844
mszoning_RL  7459.0780   1316.796      5.665      0.000    4875.982       1e+04
mszoning_RL  7459.0780   1316.796      5.665      0.000    4875.982       1e+04
==============================================================================
Omnibus:                      419.459   Durbin-Watson:                   1.972
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            39813.838
Skew:                          -0.295   Prob(JB):                         0.00
Kurtosis:                      29.007   Cond. No.                     3.34e+19
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.84e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>